In [61]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC, SVC
from sklearn.metrics import accuracy_score, confusion_matrix
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime

import json
import ast
import eli5
import shap

from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split, GroupKFold, GroupShuffleSplit
from sklearn.linear_model import Ridge, RidgeCV
import gc
from catboost import CatBoostClassifier
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import altair as alt
from  altair.vega import v3
from IPython.display import HTML
from sklearn.linear_model import LinearRegression

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats

from sklearn.ensemble import RandomForestClassifier

In [43]:
target = pd.read_csv('target.csv')
data = pd.read_csv('data.csv')
test = pd.read_csv('test.csv')
vdata = pd.read_csv('vdata.csv')
vtarget = pd.read_csv('vtarget.csv')
wdata = pd.read_csv('whole_data.csv')
wtarget = pd.read_csv('whole_target.csv', header=None)

winedata = pd.read_csv('winedata.csv', header=None)

ccdata = pd.read_csv('creditcard.csv', header=None)

irisdata = pd.read_csv('irisdata.csv', header=None)
iristarget = pd.read_csv('iristarget.csv', header=None)

print('Data is ready !!')

Data is ready !!


In [64]:
wtarget
wdata

,series_id,group_id,surface,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
0,0,13,0,-0.75853,-0.63435,-0.104880,-0.105970,0.10765000,0.0175610,0.00076741,...,-0.634350907534775,-0.104880150046894,-0.105970151606306,2.843273136958140,-0.024667685991210,1.396666825132903,0.109075662450925,10.005392360367482,0.994367888560366,91.728916749591960
1,0,13,0,-0.75853,-0.63434,-0.104900,-0.106000,0.06785100,0.0299390,0.00338550,...,-0.634341583794326,-0.104900261910056,-0.106000264656491,2.843200533913637,-0.024662137066836,1.396651391031350,0.074239905254856,9.538637785475450,0.994363618853787,128.483970348972520
2,0,13,0,-0.75853,-0.63435,-0.104920,-0.105970,0.00727470,0.0289340,-0.00597830,...,-0.634348245807504,-0.104919709860681,-0.105969706957076,2.843222148572240,-0.024728179515743,1.396676561901299,0.030427581024130,8.874696779839859,0.994372108317606,291.666194982832735
3,0,13,0,-0.75852,-0.63436,-0.104950,-0.105970,-0.01305300,0.0194480,-0.00897350,...,-0.634357036481155,-0.104949509708521,-0.105969504943420,2.843183196295201,-0.024769390879673,1.396711741369982,0.025082448350390,10.164638157632568,0.994374025455211,405.249041705865807
4,0,13,0,-0.75852,-0.63435,-0.104950,-0.105960,0.00513490,0.0076517,0.00524520,...,-0.634351732734124,-0.104950286672100,-0.105960289430926,2.843196599732560,-0.024784568729166,1.396698337932622,0.010603199231364,10.556132440723733,0.994367645994177,995.561076462551682
5,0,13,0,-0.75853,-0.63439,-0.104830,-0.105800,0.05966400,0.0130430,-0.01323100,...,-0.634389555324797,-0.104829926519489,-0.105799925839568,2.843600942756283,-0.024798982812250,1.396744462892928,0.062489776011761,10.458847316917863,0.994388134432426,167.368935919205740
6,0,13,0,-0.75853,-0.63441,-0.104810,-0.105690,0.08214000,0.0443560,-0.00269560,...,-0.634410215318864,-0.104810035572532,-0.105690035871204,2.843796286837290,-0.024904063924844,1.396788780577124,0.093390045483231,9.459259808135096,0.994398785749460,101.287666787072780
7,0,13,0,-0.75852,-0.63444,-0.104800,-0.105610,0.05621800,0.0381620,-0.02293100,...,-0.634438979791219,-0.104799831476766,-0.105609830174249,2.843932782315640,-0.024981893295424,1.396858232034151,0.071712150497667,8.609989066420990,0.994409243722121,120.063183249554513
8,0,13,0,-0.75851,-0.63445,-0.104850,-0.105590,-0.01284600,0.0390040,-0.00783060,...,-0.634447781593765,-0.104849633383413,-0.105589630795942,2.843898575175309,-0.025078849439217,1.396901700144715,0.041804904357743,7.850819721029135,0.994413266755829,187.796619598655212
9,0,13,0,-0.75851,-0.63443,-0.104890,-0.105670,-0.09008200,0.0272990,-0.00997040,...,-0.634427809324557,-0.104889637816706,-0.105669635123380,2.843723123597360,-0.025042235982047,1.396867427664848,0.094654165260489,8.871544575867274,0.994404724998831,93.725876208962305


In [76]:
wdata = wdata.drop(['series_id','group_id','surface'], axis=1)

In [11]:
n_fold = 20
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=11)
#folds = GroupShuffleSplit(n_splits=n_fold, test_size=0.2, random_state=11)
#folds = GroupKFold(n_splits=n_fold)

In [14]:
le = LabelEncoder()
le.fit(y['surface'])
# target['surface'] = le.transform(y['surface'])

# data = train_df.drop(['series_id'], axis=1)
# test = test.drop(['series_id'], axis=1)

NameError: name 'y' is not defined

In [62]:
def eval_acc(preds, dtrain):
    labels = dtrain.get_label()
    return 'acc', accuracy_score(labels, preds.argmax(1)), True

def train_model(X, X_test, y, params=None, folds=folds, model_type='lgb', plot_feature_importance=False, model=None, groups=wdata['group_id']):

    oof = np.zeros((len(X), 9))
    prediction = np.zeros((len(X_test), 9))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y, groups)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators = 10000, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='multi_logloss',
                    verbose=5000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict_proba(X_valid)
            score = accuracy_score(y_valid, y_pred_valid.argmax(1))
            print(f'Fold {fold_n}. Accuracy: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)
        
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=20000,  eval_metric='MAE', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid
        scores.append(accuracy_score(y_valid, y_pred_valid.argmax(1)))

        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction
    
    else:
        return oof, prediction

In [ ]:
params = {'num_leaves': 123,
          'min_data_in_leaf': 12,
          'objective': 'multiclass',
          'max_depth': 22,
          'learning_rate': 0.04680350949723872,
          "boosting": "gbdt",
          "bagging_freq": 5,
          "bagging_fraction": 0.8933018355190274,
          "bagging_seed": 11,
          "verbosity": -1,
          'reg_alpha': 0.9498109326932401,
          'reg_lambda': 0.8058490960546196,
          "num_class": 9,
          'nthread': -1,
          'min_split_gain': 0.009913227240564853,
          'subsample': 0.9027358830703129
         }


oof_lgb, prediction_lgb, feature_importance = train_model(X=wdata, X_test=test, y=wtarget, params=params, model_type='lgb', plot_feature_importance=False)

Fold 0 started at Fri May  3 16:04:38 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1975]	training's multi_logloss: 0.00307763	training's multi_logloss: 0.00307763	valid_1's multi_logloss: 0.0160894	valid_1's multi_logloss: 0.0160894
Fold 1 started at Fri May  3 16:12:32 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1760]	training's multi_logloss: 0.00310912	training's multi_logloss: 0.00310912	valid_1's multi_logloss: 0.0145339	valid_1's multi_logloss: 0.0145339


In [ ]:
# I use code from this kernel: https://www.kaggle.com/theoviel/deep-learning-starter
import itertools

def plot_confusion_matrix(truth, pred, classes, normalize=False, title=''):
    cm = confusion_matrix(truth, pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', size=15)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.tight_layout()

In [ ]:
plot_confusion_matrix(y['surface'], oof_lgb.argmax(1), le.classes_)

In [ ]:
model = lgb.LGBMClassifier(**params, n_estimators = 20000, n_jobs = -1)
X_train, X_valid, y_train, y_valid = train_test_split(train_df, y['surface'], test_size=0.1, stratify=y['surface'])
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=5000, early_stopping_rounds=200)

In [ ]:
eli5.show_weights(model, targets=[0, 1], feature_names=list(X_train.columns), top=40, feature_filter=lambda x: x != '<BIAS>')

In [ ]:
model = SVC(probability=True)
oof_svc, prediction_svc = train_model(X=train_df, X_test=test_df, y=y['surface'], params=None, model_type='sklearn', model=model)

In [ ]:
sub['surface'] = le.inverse_transform(prediction_lgb.argmax(1))
sub.to_csv('lgb_sub.csv', index=False)
sub['surface'] = le.inverse_transform(prediction_svc.argmax(1))
sub.to_csv('scv_sub.csv', index=False)
sub['surface'] = le.inverse_transform((prediction_lgb + prediction_svc).argmax(1))
sub.to_csv('blend.csv', index=False)

In [ ]:
# Preparing altair. I use code from this great kernel: https://www.kaggle.com/notslush/altair-visualization-2018-stackoverflow-survey

vega_url = 'https://cdn.jsdelivr.net/npm/vega@' + v3.SCHEMA_VERSION
vega_lib_url = 'https://cdn.jsdelivr.net/npm/vega-lib'
vega_lite_url = 'https://cdn.jsdelivr.net/npm/vega-lite@' + alt.SCHEMA_VERSION
vega_embed_url = 'https://cdn.jsdelivr.net/npm/vega-embed@3'
noext = "?noext"

paths = {
    'vega': vega_url + noext,
    'vega-lib': vega_lib_url + noext,
    'vega-lite': vega_lite_url + noext,
    'vega-embed': vega_embed_url + noext
}

workaround = """
requirejs.config({{
    baseUrl: 'https://cdn.jsdelivr.net/npm/',
    paths: {}
}});
"""

#------------------------------------------------ Defs for future rendering
def add_autoincrement(render_func):
    # Keep track of unique <div/> IDs
    cache = {}
    def wrapped(chart, id="vega-chart", autoincrement=True):
        if autoincrement:
            if id in cache:
                counter = 1 + cache[id]
                cache[id] = counter
            else:
                cache[id] = 0
            actual_id = id if cache[id] == 0 else id + '-' + str(cache[id])
        else:
            if id not in cache:
                cache[id] = 0
            actual_id = id
        return render_func(chart, id=actual_id)
    # Cache will stay outside and 
    return wrapped
            
@add_autoincrement
def render(chart, id="vega-chart"):
    chart_str = """
    <div id="{id}"></div><script>
    require(["vega-embed"], function(vg_embed) {{
        const spec = {chart};     
        vg_embed("#{id}", spec, {{defaultStyle: true}}).catch(console.warn);
        console.log("anything?");
    }});
    console.log("really...anything?");
    </script>
    """
    return HTML(
        chart_str.format(
            id=id,
            chart=json.dumps(chart) if isinstance(chart, dict) else chart.to_json(indent=None)
        )
    )

HTML("".join((
    "<script>",
    workaround.format(json.dumps(paths)),
    "</script>",
)))



In [ ]:
model = lgb.LGBMClassifier(**params, n_estimators = 20000, n_jobs = -1)
X_train, X_valid, y_train, y_valid = train_test_split(train_df, y['surface'], test_size=0.1, stratify=y['surface'])
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=5000, early_stopping_rounds=200)

In [ ]:
eli5.show_weights(model, targets=[0, 1], feature_names=list(X_train.columns), top=40, feature_filter=lambda x: x != '<BIAS>')

In [ ]:
model = SVC(probability=True)
oof_svc, prediction_svc = train_model(X=train_df, X_test=test_df, y=y['surface'], params=None, model_type='sklearn', model=model)

In [ ]:
sub['surface'] = le.inverse_transform(prediction_lgb.argmax(1))
sub.to_csv('lgb_sub.csv', index=False)
sub['surface'] = le.inverse_transform(prediction_svc.argmax(1))
sub.to_csv('scv_sub.csv', index=False)
sub['surface'] = le.inverse_transform((prediction_lgb + prediction_svc).argmax(1))
sub.to_csv('blend.csv', index=False)